In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

In [2]:
# Load in the dataset
import pandas as pd
heart_df = pd.read_csv("Resources/heart.csv")
heart_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
# Find out how many rows
heart_df[heart_df.columns[0]].count()

918

In [4]:
# Generate our categorical variable list
heart_cat = heart_df.dtypes[heart_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
heart_df[heart_cat].nunique()

Sex               2
ChestPainType     4
RestingECG        3
ExerciseAngina    2
ST_Slope          3
dtype: int64

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(heart_df[heart_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(heart_cat)
encode_df.head()

C:\Users\yutac\.conda\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
heart_df = heart_df.merge(encode_df,left_index=True, right_index=True)
heart_df = heart_df.drop(heart_cat,1)
heart_df.head()

C:\Users\yutac\.conda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,49,160,180,0,156,1.0,1,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,37,130,283,0,98,0.0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,48,138,214,0,108,1.5,1,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,54,150,195,0,122,0.0,0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [7]:
for col in heart_df.columns:
    print(col)

Age
RestingBP
Cholesterol
FastingBS
MaxHR
Oldpeak
HeartDisease
Sex_F
Sex_M
ChestPainType_ASY
ChestPainType_ATA
ChestPainType_NAP
ChestPainType_TA
RestingECG_LVH
RestingECG_Normal
RestingECG_ST
ExerciseAngina_N
ExerciseAngina_Y
ST_Slope_Down
ST_Slope_Flat
ST_Slope_Up


In [8]:
# Remove loan status target from features data
y = heart_df.HeartDisease.values
X = heart_df.drop(columns=["HeartDisease"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create the SVM model
svm = SVC(kernel='linear')

In [10]:
# Train the model
svm.fit(X_train, y_train)

SVC(kernel='linear')

In [11]:
# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.857


In [12]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
22/22 [==============================] - 1s 2ms/step - loss: 0.7954 - accuracy: 0.4898
Epoch 2/50
22/22 [==============================] - 0s 2ms/step - loss: 0.6849 - accuracy: 0.5843
Epoch 3/50
22/22 [==============================] - 0s 2ms/step - loss: 0.6027 - accuracy: 0.7166
Epoch 4/50
22/22 [==============================] - 0s 2ms/step - loss: 0.5363 - accuracy: 0.7834
Epoch 5/50
22/22 [==============================] - 0s 1ms/step - loss: 0.4846 - accuracy: 0.8169
Epoch 6/50
22/22 [==============================] - 0s 1ms/step - loss: 0.4460 - accuracy: 0.8358
Epoch 7/50
22/22 [==============================] - 0s 1ms/step - loss: 0.4178 - accuracy: 0.8416
Epoch 8/50
22/22 [==============================] - 0s 1ms/step - loss: 0.3978 - accuracy: 0.8445
Epoch 9/50
22/22 [==============================] - 0s 1ms/step - loss: 0.3837 - accuracy: 0.8445
Epoch 10/50
22/22 [==============================] - 0s 1ms/step - loss: 0.3729 - accuracy: 0.8445
Epoch 11/50
22/22 [

In [14]:
# Trying Logistic regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.8826086956521739


C:\Users\yutac\.conda\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 86,  17],
       [ 10, 117]], dtype=int64)

In [18]:
# List the features sorted in descending order by feature importance
X

array([[ 40., 140., 289., ...,   0.,   0.,   1.],
       [ 49., 160., 180., ...,   0.,   1.,   0.],
       [ 37., 130., 283., ...,   0.,   0.,   1.],
       ...,
       [ 57., 130., 131., ...,   0.,   1.,   0.],
       [ 57., 130., 236., ...,   0.,   1.,   0.],
       [ 38., 138., 175., ...,   0.,   0.,   1.]])